In [4]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
drive_path = "/content/drive/MyDrive/"

In [6]:
import os
import numpy as np
from glob import glob
from tqdm import tqdm
import time
import tensorflow as tf
import cv2

In [7]:
from google.colab.patches import cv2_imshow

In [8]:
os.environ["PYTHONASHSEED"] = "42"
np.random.seed(42)
tf.random.set_seed(42)

In [9]:
height = 720
width = 1280

In [10]:
dataset_path = "datasets/non-augmented/test"
save_path = "predictions"
model_path = "unet-original.h5"

In [11]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [12]:
create_dir(os.path.join(drive_path,save_path))

In [13]:
model = tf.keras.models.load_model(os.path.join(drive_path,model_path))
model.summary()

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 720, 1280, 3)]       0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 720, 1280, 64)        1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 720, 1280, 64)        256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 720, 1280, 64)        0         ['batch_normalization[0][0]

In [14]:
test_x = sorted(glob(os.path.join(drive_path,"images","*")))

In [15]:
print(len(test_x))

5


In [16]:
time_taken = []
print(test_x)
i = 0
for x in tqdm(test_x):
    name = x.split("/")[-1]
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    img = x.copy()
    x = x/255.0
    x = np.expand_dims(x, axis=0)
    start_time = time.time()
    p = model.predict(x)[0]
    stop_time = time.time()
    time_taken.append(stop_time-start_time)
    p = p>0.56
    p = p*255
    p = p.astype(np.uint8)
    mask = np.zeros((height, width, 4), dtype=np.uint8)
    mask[:, :, 0] = 180
    mask[:, :, 1] = 20
    mask[:, :, 2] = 102
    mask[:, :, 3] = 45
    mask = np.where(p>0,mask, p)
    result = cv2.addWeighted(img, 1, mask[:, :, :3], 1, 0, dtype=cv2.CV_8U)
    cv2_imshow(result)



Output hidden; open in https://colab.research.google.com to view.